In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
import pandas as pd

# dataset

In [2]:
labelDf = pd.read_csv("train_labels.csv")
labelDf = labelDf.set_index('planet_id')

mean = np.mean(labelDf.mean())
std = np.std(labelDf.std())
max = np.max(labelDf.max())
min = np.min(labelDf.min())
mean, std, max, min

for col in labelDf.columns:
    labelDf.loc[:,col] = (labelDf[col] - mean) / (std)

In [3]:
tf.random.set_seed(42)
files = glob.glob(os.path.join('train/', '*/*'))
stars = []
for file in files:
    file_name = file.split('\\')[1]
    stars.append(file_name)
stars = np.unique(stars)

import random
random.seed(42)

def split_star_list(file_list, test_ratio=0.2):
    random.shuffle(file_list)
    split_index = int(len(file_list) * (1 - test_ratio))
    train_files = file_list[:split_index]
    test_files = file_list[split_index:]
    return train_files, test_files

train_stars, test_stars = split_star_list(stars)

def preprocess_data(features, labels):
    # Perform any necessary preprocessing here
    return features, labels

def load_npz(star):
    integer_value = tf.strings.to_number(star, out_type=tf.int64)
    python_int = integer_value.numpy()

    file_path = 'train/'+str(python_int)+'/combined.npz'
    try:
        with np.load(file_path) as data:
            features = data['a'][0,:,0:283,:]
            labels = labelDf.loc[python_int].to_numpy()

            features, labels = preprocess_data(features,labels)
            return features, labels
    except Exception as e:
        print("Error loading file:", e, python_int)
    

def create_dataset(star_list, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(star_list)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(star_list))
    def load_and_process(x):
        features, labels = tf.py_function(
            func=load_npz,
            inp=[x],
            Tout=[tf.float64, tf.float32]
        )
        return features, labels

    dataset = dataset.map(load_and_process, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (tf.ensure_shape(x,tf.TensorShape([5625, 283, 4])), tf.ensure_shape(y, tf.TensorShape([283]))))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


In [4]:
tf.random.set_seed(42)
batch_size = 12

train_dataset = create_dataset(train_stars, batch_size, shuffle=True)
test_dataset = create_dataset(test_stars, batch_size, shuffle=False)

# model

In [ ]:
tf.random.set_seed(42)

class ReduceDim(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        x = tf.squeeze(x, axis=-1)
        return x
    
class Reshape1(tf.keras.layers.Layer):
    def __init__(self, timepoints, representations, wavelengths, **kwargs):
        super().__init__(**kwargs)
        self.timepoints = timepoints
        self.wavelengths = wavelengths
        self.representations = representations
    def call(self, x):
        x = tf.transpose(x, perm=[0,2,1,3])
        #x = tf.reshape(x, [-1, self.timepoints, tf.cast(self.wavelengths * self.representations, tf.int32)])
        return x
    
class Reshape11(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        x = tf.transpose(x, perm=[0,2,1])
        #x = tf.reshape(x, [-1, self.timepoints, tf.cast(self.wavelengths * self.representations, tf.int32)])
        return x

class Reshape2(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x_pred, x_confidence):
        x = tf.concat([x_pred, x_confidence], axis = -1)
        
        return x
    
class Reshape22(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x_pred, x_confidence):
        x_pred = tf.expand_dims(x_pred, axis=-1)
        x_confidence = tf.expand_dims(x_confidence, axis=-1)
        x = tf.concat([x_pred, x_confidence], axis = -1)
        
        return x

class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim//num_heads)
        self.ffn1 = tf.keras.layers.Dense(embed_dim)
        self.ffn2 = tf.keras.layers.Dense(feed_forward_dim)
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    def call(self, x):
        residual = x
        x = self.att(x, x)
        x = x + residual
        residual = x
        #x = self.ffn1(x)
        x = self.ffn2(x)
        x = self.layer_norm(x + residual)
        return x
    

timepoints = 5625
representations = 4
wavelengths = 283
targetWavelengths = 283
def buildModel_bad(outputDim = 283):
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp

    # plan:
    # 1. use cnn filter so go from timepoints, wavelengths, representations -> timepoints, wavelengths (283 1d filter)
    # 2. transpose, timepoints,wavelengths -> wavelengths,timepoints

    # edit: doesn't make sense since transformer should not learn similarities between wavelengths but rather similarities between timepoints

       # batch, wavelengths, time, representation

    x = Reshape1(timepoints, representations, wavelengths)(x) #make to [batch_size, wavelengths, time,repr]
    # Use a 1D Convolutional layer with kernel size of 1 to reduce the last dimension
    x = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), padding='valid')(x)
    x = ReduceDim()(x)
    dim = timepoints #int(timepoints/4)# x.shape[2]
    x = tf.keras.layers.Dense(dim)(x)

    for i in range(5):
        x = TransformerEncoder(embed_dim=dim, num_heads=4, feed_forward_dim=int(dim/(2)))(x)
        dim = int(dim/2)

    x_pred = tf.keras.layers.Dense(1, activation='relu')(x)
    x_confidence = tf.keras.layers.Dense(1, activation='relu')(x)
    x = Reshape2()(x_pred, x_confidence)

    model = tf.keras.Model(inp, x)
    return model


def buildModel(outputDim = 283):
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp

    # plan:
    # 1. use cnn filter so go from timepoints, wavelengths, representations -> timepoints, wavelengths (283 1d filter)
    # Use a 1D Convolutional layer with kernel size of 1 to reduce the last dimension
    x = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), padding='valid')(x)
    x = ReduceDim()(x)
    dim = wavelengths #int(timepoints/4)# x.shape[2]
    #x = tf.keras.layers.Dense(dim)(x)
    #x = tf.keras.layers.Conv2D(filters=dim, kernel_size=(3, 3), padding='valid')(x)

    #x = tf.keras.layers.Conv1D(filters=dim, kernel_size=8, padding='same', activation='relu')(x)

    for i in range(5):
        x = TransformerEncoder(embed_dim=dim, num_heads=4, feed_forward_dim=dim)(x)
        #dim = int(dim/2)

    x = Reshape11()(x)  # reshape to wavelengths, timestamps
    #x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(2000)(x)
    x = tf.keras.layers.Dense(200)(x)
    x = tf.keras.layers.Dense(5)(x)
    x = tf.keras.layers.Flatten()(x)

    x_pred = tf.keras.layers.Dense(283, activation='relu')(x)
    x_confidence = tf.keras.layers.Dense(283, activation='relu')(x)
    x = Reshape22()(x_pred, x_confidence)

    model = tf.keras.Model(inp, x)
    return model

def buildModel(outputDim = 283):
    inp = tf.keras.Input(shape=(timepoints, wavelengths, representations))
    x = inp[:,:,:,0]
    #x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    # plan:
    # 1. use cnn filter so go from timepoints, wavelengths, representations -> timepoints, wavelengths (283 1d filter)
    # Use a 1D Convolutional layer with kernel size of 1 to reduce the last dimension
    #x = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), padding='valid')(x)
    #x = ReduceDim()(x)
    x = tf.keras.layers.Flatten()(x)
    #x = tf.keras.layers.Dense(dim)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x = tf.keras.layers.Dense(283*4, activation='relu')(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x = tf.keras.layers.Dense(283*4, activation='relu')(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x = tf.keras.layers.Dense(283*4, activation='relu')(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    x_pred = tf.keras.layers.Dense(283, activation='linear')(x)
    x_confidence = tf.keras.layers.Dense(283, activation='linear')(x)
    x = Reshape22()(x_pred, x_confidence)

    model = tf.keras.Model(inp, x)
    return model


model = buildModel()
model.summary()

In [ ]:
batch = next(iter(train_dataset))
out = model(batch[0])
test_batch = next(iter(test_dataset))
batch[0].dtype ,batch[1].dtype, out.dtype,batch[0].shape ,batch[1].shape, out.shape

In [ ]:
def log_likelihood_zScoreTarget(y_trueZScore, y_pred):
    # stdDev_zScorePred = 1/n * sqrt((y_zScore - y_zScoreMean)^2) = 1/n *sqrt(sum( (y-mean)/std - (y_mean-mean)/std )^2) = 1/n * sqrt(sum( (y-y_mean)/std )^2 )) = 1/std * 1/n * sqrt(sum(y-y_mean)^2) = stdDev / std
    # stdDev_zScorePred = stdDev_pred / std
    # y_pred contains 1. y_zScore 2. log(stdDev_zScore)

    y_true = y_trueZScore * std + mean   # y_zScore = (y - mean) / std -> y = y_zScore *std + mean

    y_predZScore = y_pred[:, :,0]
    log_sigma = y_pred[:, :,1]  # Log of the standard deviation / we predict log(stdDev_zScore) = log(stdDev / std) = log(stdDev) - log(std) -> log(stdDev) = log(stdDev_zScore) + log(std)

    y_pred0 = y_predZScore * std + mean
    stdDev = tf.exp(log_sigma)*std  # Exponentiate to get variance + scale back from zscore 
    logStdDev = log_sigma + tf.math.log(std)


    L_pred = -0.5*(tf.math.log(2*np.pi) + logStdDev + tf.square(y_true - y_pred0) / stdDev)
    L_ref = -0.5*(tf.math.log(2*np.pi) +  tf.math.log(std*std) + tf.square(y_trueZScore))   # ( (y_true - mean)/std )^2 = y_trueZScore^2  (y_true = y_trueZScore * std + mean)
    L_ideal = -0.5*(tf.math.log(2*np.pi) + tf.math.log(1e-10))

    L = (L_pred -L_ref) / (L_ideal - L_ref)
    
    return tf.reduce_mean(L)

def log_loss_zScoreTarget(y_trueZScore, y_pred):
    # stdDev_zScorePred = 1/n * sqrt((y_zScore - y_zScoreMean)^2) = 1/n *sqrt(sum( (y-mean)/std - (y_mean-mean)/std )^2) = 1/n * sqrt(sum( (y-y_mean)/std )^2 )) = 1/std * 1/n * sqrt(sum(y-y_mean)^2) = stdDev / std
    # stdDev_zScorePred = stdDev_pred / std
    # y_pred contains 1. y_zScore 2. log(stdDev_zScore)

    y_true = y_trueZScore * std + mean   # y_zScore = (y - mean) / std -> y = y_zScore *std + mean

    y_predZScore = y_pred[:, :,0]
    log_sigma = y_pred[:, :,1]  # Log of the standard deviation / we predict log(stdDev_zScore) = log(stdDev / std) = log(stdDev) - log(std) -> log(stdDev) = log(stdDev_zScore) + log(std)

    y_pred0 = y_predZScore * std + mean
    stdDev = tf.exp(log_sigma)*std  # Exponentiate to get variance + scale back from zscore 
    logStdDev = log_sigma + tf.math.log(std)


    L_pred = -0.5*(tf.math.log(2*np.pi) + logStdDev + tf.square(y_true - y_pred0) / stdDev)
    L_ref = -0.5*(tf.math.log(2*np.pi) +  tf.math.log(std*std) + tf.square(y_trueZScore))   # ( (y_true - mean)/std )^2 = y_trueZScore^2  (y_true = y_trueZScore * std + mean)
    L_ideal = -0.5*(tf.math.log(2*np.pi) + tf.math.log(1e-10))

    L = (L_pred -L_ref) / (L_ideal - L_ref)
    loss = -(L - 1)
    #print(L)
    
    return tf.reduce_mean(loss)
log_likelihood_zScoreTarget(batch[1], out),log_loss_zScoreTarget(batch[1], out)

In [ ]:
tf.random.set_seed(42)

def loss_fn0(y_true_zScore, y_pred):
    y_predZScore = y_pred[:, :,0]  # y_zScore = (y - mean)/std
    logConfidence = y_pred[:, :,1] # logStdDev = log(stdDev / std)

    loss = tf.math.abs(y_true_zScore-y_predZScore)
    loss_2 = tf.math.abs(loss-tf.exp(logConfidence))
    return tf.reduce_mean(loss+0.0001*loss_2)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss=loss_fn0,metrics=[log_likelihood_zScoreTarget], optimizer=optimizer)

history = model.fit(#train_dataset, 
                    batch[0],batch[1], #verbose=2,
                    #validation_data=test_dataset,
                    epochs=400, batch_size=batch_size,
                    #callbacks=[validation_callback(val_metric_list), lr_callback, WeightDecayCallback()]
                    )

# fcn loss on one batch ~1e-8
#gaussian_log_likelihood: -0.1196 - loss: 309017632.0000 - val_gaussian_log_likelihood: 0.8712 - val_loss: 1794403072.0000
#gaussian_log_likelihood: -24139.4531 - loss: 117853856.0000 - val_gaussian_log_likelihood: -18151.4355 - val_loss: 987686784.0000

In [ ]:
model.save_weights('a.weights.h5')

In [ ]:
history = model.fit(#train_dataset, 
                    batch[0],batch[1], #verbose=2,
                    validation_data=test_dataset,
                    epochs=200, batch_size=batch_size,
                    #callbacks=[validation_callback(val_metric_list), lr_callback, WeightDecayCallback()]
                    )

In [ ]:
pred = model.predict(batch[0])
pred[:,:,0],np.exp(pred[:,:,1]), batch[1]

In [ ]:
pred[:,0,0]

In [ ]:
for x,y in test_dataset:
    print(x.shape, y.shape)

In [ ]:
np.mean(np.abs(pred[:,:,0]-batch[1]))

In [ ]:
gaussian_log_likelihood(batch[1],pred)

In [ ]:
loss_fn0(batch[1],np.zeros((batch[1].shape[0], batch[1].shape[1],2)))

In [ ]:
pred.sum()

In [ ]:
pred.shape, batch[1].shape

In [ ]:
pred

In [ ]:
batch[1]